In [2]:
import pandas as pd

In [3]:
expert_all = pd.read_csv('../data/expert/expert_table.csv')
expert_all.head()

,#,context,disease_name,drug_name,workers_answers,medical_expert1,medical_expert2,medical_expert3,do_id,drug_brand_name,drug_id,label_id,sheet,expert_consensus,inter_agree_experts
0,1.0,Nisoldipine extended-release tablets are indic...,HYPERTENSION,NISOLDIPINE,indication_treatment\nindication_treatment\nin...,Indication: Treatment,Indication: Treatment,Indication: Treatment,DOID_10763,NaN,DB00401,ce733b39-7857-4538-92d6-1c68a4e4eb75.xml,Indication_treatment,Indication: Treatment,3
1,2.0,"[None, 'Bisoprolol fumarate tablets, USP are i...",HYPERTENSION,BISOPROLOL,indication_treatment\nindication_treatment\nin...,Indication: Treatment,Indication: Treatment,Indication: Treatment,DOID_10763,NaN,DB00612,c818c38d-ee37-4ceb-b078-611483b4f743.xml,Indication_treatment,Indication: Treatment,3
2,3.0,Pindolol tablets are indicated in the manageme...,HYPERTENSION,PINDOLOL,indication_treatment\nindication_treatment\nin...,Indication: Treatment,Indication: Treatment,Indication: Treatment,DOID_10763,NaN,DB00960,d4078b63-30ad-435b-a081-4d90562962b5.xml,Indication_treatment,Indication: Treatment,3
3,4.0,Eprosartan mesylate tablets are indicated for ...,HYPERTENSION,EPROSARTAN,indication_treatment\nindication_treatment\nin...,Indication: Treatment,Indication: Treatment,Indication: Treatment,DOID_10763,NaN,DB00876,ab531d28-345b-4113-9fde-00cd63ad5b1a.xml,Indication_treatment,Indication: Treatment,3
4,5.0,"[None, 'Labetalol hydrochloride tablets, USP a...",HYPERTENSION,LABETALOL,indication_treatment\nindication_treatment\nin...,Indication: Treatment,Indication: Treatment,Indication: Treatment,DOID_10763,NaN,DB00598,78765cc3-27dc-0373-e053-2991aa0a7fe1.xml,Indication_treatment,Indication: Treatment,3


In [4]:
expert_all.expert_consensus.unique()

array(['Indication: Treatment', 'No consensus',
       'Indication: Symptomatic Relief', 'Contraindication', 'Effect'],
      dtype=object)

In [5]:
expert4 = pd.read_excel('../data/expert/Medical Expert Annotation 4.xlsx', sheet_name='Sheet1')
expert4.head()

,#,context,disease_name,drug_name,workers_answers,Medical Expert,do_id,drug_brand_name,drug_id,label_id,sheet
0,7,\n \nHypertension\nTimolol...,HYPERTENSION,TIMOLOL ANHYDROUS,indication_treatment\nindication_treatment\nin...,indication_treatment,DOID_10763,NaN,DB00373,87fe4811-1f32-ae9e-e053-2a95a90a9f1d.xml,Indication_treatment
1,17,Fosinopril sodium tablets are indicated for th...,HYPERTENSION,FOSINOPRIL,indication_treatment\nindication_treatment\nin...,indication_treatment,DOID_10763,NaN,DB00492,29914956-c794-4806-b038-f0784b9db3d3.xml,Indication_treatment
2,19,"['\n ', '1.1 Supraventricu...",HYPERTENSION,ESMOLOL,indication_treatment\nindication_treatment\nin...,indication_treatment,DOID_10763,NaN,DB00187,e296498d-530b-4346-af09-43008d65c46a.xml,Indication_treatment
3,20,['Amiloride HCl tablets are indicated as adjun...,HYPERTENSION,AMILORIDE,indication_treatment\nindication_treatment\nin...,NaN,DOID_10763,NaN,DB00594,62d4c259-7915-719c-e053-2a91aa0a35de.xml,Indication_treatment
4,21,['Carvedilol is an alpha/beta-adrenergic block...,HYPERTENSION,CARVEDILOL,indication_treatment\nindication_treatment\ni_...,indication_treatment,DOID_10763,NaN,DB01136,674ef26b-f8c1-2373-e053-2991aa0a20f6.xml,Indication_treatment


In [6]:
len(expert4)

181

In [7]:
expert4.rename(columns={'Medical Expert':'medical_expert4'}, inplace=True)

In [8]:
expert4.medical_expert4.replace({'indication_treatment':'Indication: Treatment',
                                'indication_symptomatic_relief':'Indication: Symptomatic Relief',
                                'contraindication':'Contraindication',
                                'contraindicated':'Contraindication',
                                'effect':'Effect'}, inplace=True)

In [9]:
expert4.medical_expert4.unique()

array(['Indication: Treatment', nan, 'Indication: Symptomatic Relief',
       'Effect', 'Contraindication'], dtype=object)

In [10]:
match =expert_all.merge(expert4[['medical_expert4','label_id','do_id']], how='outer', on=['label_id','do_id'])
len(match)

359

In [11]:
match.inter_agree_experts.value_counts()

2    181
3    150
0     28
Name: inter_agree_experts, dtype: int64

In [12]:
consesus = [] 
resolved = 0
for i, row in match.iterrows():
    if (row['inter_agree_experts']) == 2:
        if str(row['medical_expert4']) != 'nan':
            consesus.append(row['medical_expert4'])
            #print (row['medical_expert4'])
            resolved +=1
            #print ('droping', row['label_id'],row['do_id'])
        else:
            consesus.append(row['expert_consensus'])
    else:
        consesus.append(row['expert_consensus']) 
print ('resolved',resolved)

resolved 143


In [13]:
match.expert_consensus = consesus

In [14]:
match.expert_consensus.value_counts()

Indication: Treatment             118
Contraindication                  116
Indication: Symptomatic Relief     58
Effect                             39
No consensus                       28
Name: expert_consensus, dtype: int64

In [16]:
meta_expert = pd.read_excel('../data/expert/experts_all_disagree_review.xlsx', sheet_name='experts_all_disagree')

In [17]:
meta_expert['Final \n(closest option)'].replace({'Indication_treatment':'Indication: Treatment',
                                'Indication: Symptomatic relief':'Indication: Symptomatic Relief',
                                'Treatment: Symptomatic relief':'Indication: Symptomatic Relief',
                                 'contraindication':'Contraindication',
                                'contraindicated':'Contraindication',
                                'effect':'Effect'}, inplace=True)

In [18]:
meta_expert['Final \n(closest option)'].unique()

array(['Indication: Treatment', 'Indication: Symptomatic Relief',
       'Contraindication'], dtype=object)

In [19]:
match= match.merge(meta_expert[['Final \n(closest option)','label_id','do_id']], how='outer', on=['label_id','do_id'])


In [20]:
match.head()

,#,context,disease_name,drug_name,workers_answers,medical_expert1,medical_expert2,medical_expert3,do_id,drug_brand_name,drug_id,label_id,sheet,expert_consensus,inter_agree_experts,medical_expert4,Final \n(closest option)
0,1.0,Nisoldipine extended-release tablets are indic...,HYPERTENSION,NISOLDIPINE,indication_treatment\nindication_treatment\nin...,Indication: Treatment,Indication: Treatment,Indication: Treatment,DOID_10763,NaN,DB00401,ce733b39-7857-4538-92d6-1c68a4e4eb75.xml,Indication_treatment,Indication: Treatment,3,NaN,NaN
1,2.0,"[None, 'Bisoprolol fumarate tablets, USP are i...",HYPERTENSION,BISOPROLOL,indication_treatment\nindication_treatment\nin...,Indication: Treatment,Indication: Treatment,Indication: Treatment,DOID_10763,NaN,DB00612,c818c38d-ee37-4ceb-b078-611483b4f743.xml,Indication_treatment,Indication: Treatment,3,NaN,NaN
2,3.0,Pindolol tablets are indicated in the manageme...,HYPERTENSION,PINDOLOL,indication_treatment\nindication_treatment\nin...,Indication: Treatment,Indication: Treatment,Indication: Treatment,DOID_10763,NaN,DB00960,d4078b63-30ad-435b-a081-4d90562962b5.xml,Indication_treatment,Indication: Treatment,3,NaN,NaN
3,4.0,Eprosartan mesylate tablets are indicated for ...,HYPERTENSION,EPROSARTAN,indication_treatment\nindication_treatment\nin...,Indication: Treatment,Indication: Treatment,Indication: Treatment,DOID_10763,NaN,DB00876,ab531d28-345b-4113-9fde-00cd63ad5b1a.xml,Indication_treatment,Indication: Treatment,3,NaN,NaN
4,5.0,"[None, 'Labetalol hydrochloride tablets, USP a...",HYPERTENSION,LABETALOL,indication_treatment\nindication_treatment\nin...,Indication: Treatment,Indication: Treatment,Indication: Treatment,DOID_10763,NaN,DB00598,78765cc3-27dc-0373-e053-2991aa0a7fe1.xml,Indication_treatment,Indication: Treatment,3,NaN,NaN


In [21]:
indx = match[~match['Final \n(closest option)'].isna()].index
indx
match.loc[indx,'expert_consensus'] = match.loc[indx,'Final \n(closest option)']

In [22]:
match.expert_consensus.value_counts()

Contraindication                  133
Indication: Treatment             122
Indication: Symptomatic Relief     65
Effect                             39
Name: expert_consensus, dtype: int64

In [23]:
len(match[match.expert_consensus=='No consensus'])

0

In [24]:
match = match[match.expert_consensus!='No consensus']
len(match)

359

In [25]:
match.to_csv('../data/expert/expert_resolved_all.csv', index=False)